## ReAct (Reason-Action prompting)

In [19]:
import lionagi as li

### Set up

Let's set up a simple multi-step accounting problem and only provides the most trivial tools for LLM to solve it. 

for tools, we will only provide the following 
- Two numbers addition, $a+b$
- Two numbers multiplication, $a*b$
- Two numbers substraction, $a-b$

In [20]:
instruction = """
solve the following accounting problem
"""

context = """
I have 730_000 trees, with average 123 apples per tree, each weigh 0.4 lbs. 20 percent are bad and sold for 0.1 dollar per lbs, 30 percent are sold to brewery for 0.3 dollar per apple, the remaining are sold as fruit for 60 cents per apple. The variable cost of producing apples is 10 cents each apple. The fixed cost is 10 million dollars. 50 million dollars was borrowed from bank, and I repaid in full to bank with 10 percent extra in interest. I have to pay 10 percent of all proceeds as sales tax. And, I need to pay 35 percent on the remaining as business income tax. what is my accounting profit, and what is my cash flow?
"""

In [21]:
async def multiply(number1:float, number2:float):
    '''
    multiply two numbers.

    Args:
        number1: First number to multiply.
        number2: Second number to multiply.
    
    Returns:
        The product of number1, number2

    '''
    return number1*number2

tool_multiply = li.func_to_tool(multiply)


async def sum(number1:float, number2:float):
    '''
    add two numbers.


    Args:
        number1: First number to add.
        number2: Second number to add.
    
    Returns:
        The sum of number1, number2
    '''
    return number1+number2


async def minus(number1:float, number2:float):
    '''
    number 1 subtract number 2. i.e. a-b

    Args:
        number1: First number
        number2: Second number
    
    Returns:
        number 1 minus number 2.

    '''
    return number1-number2

async def division(number1:float, number2:float):
    '''
    number 1 divide by number 2. i.e. a/b

    Args:
        number1: First number
        number2: Second number
    
    Returns:
        number 1 divide number 2.

    '''
    return number1/number2


funcs = [multiply, sum, minus, division]
tools = li.lcall(funcs, li.func_to_tool)

In [22]:
session = li.Session(system='Think step by step, perform as a helpful math teacher,')
session.register_tools(tools)

In [23]:
# answer
num_apples = 730_000*123

bad_apple_income = 0.4*num_apples*.2*.1
brewery_income = num_apples*.3*.3
fruit_income = num_apples*.5*.6

revenue = bad_apple_income+brewery_income+fruit_income

variable_cost = num_apples*0.1
fixed_cost = 10_000_000
sales_tax = revenue * 0.1

cost = variable_cost + fixed_cost + sales_tax

margin = revenue - cost
biz_income_tax = 0.35*margin

operating_income = margin - biz_income_tax
bank_repayment = 50_000_000*1.1
cashflow = operating_income - bank_repayment

In [24]:
response = await session.ReAct(instruction=instruction, context=context, num_rounds=4, temperature=0.2)

In [25]:
session.messages

,node_id,role,sender,timestamp,content
0,2a547318e572eb1ba670b410a50df6a0,system,system,2024-02-11T16:56:28.638022,"{""system_info"": ""Think step by step, perform a..."
1,892dfc0d827d8145dc4d506233b5c348,user,user,2024-02-11T16:56:28.644800,"{""instruction"": {""Notice"": ""you have 8 step le..."
2,fba64ef5f5f15726aeca189e0136079b,assistant,assistant,2024-02-11T16:56:47.068927,"{""response"": ""To solve this accounting problem..."
3,6b93d2263acb71adac7ebe4aa1919e40,user,user,2024-02-11T16:56:47.069624,"{""instruction"": ""\n you have 7 ..."
4,0a61df0f3a0d1b17e9fc4995195fcd15,assistant,action_request,2024-02-11T16:56:48.400660,"{""action_request"": [{""action"": ""action_multiply"",..."
5,48c81da957c40f97ce938bfa8a25a3d0,assistant,action_response,2024-02-11T16:56:48.402180,"{""action_response"": {""function"": ""multiply"", ""..."
6,a43e22c53ecdf5e6e57a92f689d48999,user,user,2024-02-11T16:56:48.402881,"{""instruction"": {""Notice"": ""you have 6 step le..."
7,e9f38ec70a98516477b2ddc5acb411a8,assistant,assistant,2024-02-11T16:57:00.790713,"{""response"": ""The total number of apples has b..."
8,c1f10c6380c1f31a413822c78db4d516,user,user,2024-02-11T16:57:00.792217,"{""instruction"": ""\n you have 5 ..."
9,eceea489bbab3b0603bbbfbdb4fda50a,assistant,action_request,2024-02-11T16:57:04.504622,"{""action_request"": [{""action"": ""action_multiply"",..."


In [26]:
from IPython.display import Markdown

Markdown(session.last_response_content['response'])

With all the necessary calculations completed, let's summarize our findings and calculate the accounting profit and cash flow.

### Revenue Calculation:
1. **Revenue from bad apples**: $718,320
2. **Revenue from apples sold to the brewery**: $8,081,100
3. **Revenue from apples sold as fruit**: $26,937,000

### Total Revenue:
- **Total Revenue** = Revenue from bad apples + Revenue from brewery + Revenue from fruit
- **Total Revenue** = $718,320 + $8,081,100 + $26,937,000 = $35,736,420

### Costs and Expenses:
- **Variable Cost**: 10 cents per apple for 89,790,000 apples
- **Fixed Cost**: $10 million
- **Repayment to Bank**: 50 million * 10% interest = 55 million
- **Sales Tax**: 10% of all proceeds
- **Business Income Tax**: 35% on the remaining profit

### Variable Cost Calculation:
- **Variable Cost** = $0.10 * 89,790,000 = $8,979,000

### Profit Before Taxes:
- **Profit Before Taxes** = Total Revenue - Variable Cost - Fixed Cost
- **Profit Before Taxes** = $35,736,420 - $8,979,000 - $10,000,000 = $16,757,420

### Sales Tax:
- **Sales Tax** = 10% of Total Revenue = $3,573,642

### Profit After Sales Tax:
- **Profit After Sales Tax** = Profit Before Taxes - Sales Tax = $16,757,420 - $3,573,642 = $13,183,778

### Business Income Tax:
- **Business Income Tax** = 35% of Profit After Sales Tax = $4,614,322.3

### Accounting Profit:
- **Accounting Profit** = Profit After Sales Tax - Business Income Tax = $13,183,778 - $4,614,322.3 = $8,569,455.7

### Cash Flow Calculation:
- **Cash Flow** = Accounting Profit + Fixed Cost + Repayment to Bank
- **Cash Flow** = $8,569,455.7 + $10,000,000 + $55,000,000 = $73,569,455.7

### Final Result:
- **Accounting Profit**: $8,569,455.7
- **Cash Flow**: $73,569,455.7

These calculations provide a comprehensive view of the financial outcome of the apple sales, including the accounting profit and cash flow.

In [27]:
print(f"Answers: \nrevenue: {int(revenue):,}, cost: {int(cost):,}, \nmargin: {int(margin):,}, operating_income: {int(operating_income):,}, \nbank_repayment: {int(bank_repayment):,}, cashflow: {int(cashflow):,}")

Answers: 
revenue: 35,736,420, cost: 22,552,642, 
margin: 13,183,778, operating_income: 8,569,455, 
bank_repayment: 55,000,000, cashflow: -46,430,544
